In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load data
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Merge data
movie_ratings = pd.merge(movies, ratings, on='movieId',how='left')
movie_ratings.to_csv('merged_movieratings.csv')

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

movie_ratings = pd.read_csv("merged_movieratings.csv")
movie_ratings.userId.nunique(dropna = True)

283228

In [3]:
movie_ratings.genres.nunique(dropna = True)

1643

In [4]:
# Load dataset
df = movie_ratings

# Count number of movies in each genre
genre_counts = df['genres'].value_counts()
genre_counts
# Select top 200 genres with highest number of movies
top_100_genres =  genre_counts.reset_index()[:100]['genres'].tolist()
top_100_genres
# Filter dataframe based on selected genres
new_df = df[df['genres'].isin(top_100_genres)]
new_df

,Unnamed: 0,movieId,title,genres,userId,rating,timestamp
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09
...,...,...,...,...,...,...,...
27757646,27757646,193874,Blondie's Big Moment (1947),Comedy,181577.0,5.0,1.537928e+09
27757648,27757648,193878,Les tribulations d'une caissière (2011),Comedy,176871.0,2.0,1.537875e+09
27757649,27757649,193880,Her Name Was Mumu (2016),Drama,81710.0,2.0,1.537886e+09
27757651,27757651,193886,Leal (2018),Action|Crime|Drama,206009.0,2.5,1.537918e+09


In [5]:
new_df.userId.nunique(dropna = True)

281455

In [6]:
new_df.genres.nunique(dropna = True)

100

In [7]:
new_df['movieId'].nunique()

40087

In [21]:
user_unique = new_df['userId'].unique()
# Select top 1000 users with highest number of movies
top_100_users = user_unique[:100].tolist()
top_100_users
# Filter dataframe based on selected genres
new_new_df = new_df[new_df['userId'].isin(top_100_users)]
new_new_df

,Unnamed: 0,movieId,title,genres,userId,rating,timestamp
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09
...,...,...,...,...,...,...,...
27739269,27739269,185029,A Quiet Place (2018),Drama|Horror|Thriller,214.0,3.0,1.524349e+09
27746563,27746563,187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi,321.0,3.0,1.527871e+09
27749885,27749885,188213,The Escape (2018),Drama,214.0,4.0,1.533588e+09
27752407,27752407,189333,Mission: Impossible - Fallout (2018),Action|Adventure|Thriller,214.0,3.0,1.533422e+09


In [22]:
new_new_df['movieId'].nunique()

3952

In [23]:
new_new_df.userId.nunique(dropna = True)

100

In [24]:
new_new_df.genres.nunique(dropna = True)

100

In [25]:
new_new_df.to_csv("top100_100.csv")

In [26]:
# Create mapping of movieId to row index in sparse matrix
movie_id_map = dict(zip(sorted(new_new_df['movieId'].unique()), range(len(new_new_df['movieId'].unique()))))

# Create mapping of userId to column index in sparse matrix
user_id_map = dict(zip(sorted(new_new_df['userId'].unique()), range(len(new_new_df['userId'].unique()))))

In [27]:
def movie_id_vectors(df1):
    temp = [0 for i in range(new_new_df['movieId'].nunique())]
    df1 = df1.reset_index()
    for r in range(len(df1)):
        temp[movie_id_map[df1.iloc[r]['movieId']]]= df1.iloc[r]['rating']
    return temp

In [28]:
gbtest = new_new_df.groupby("userId")
ab = pd.DataFrame(gbtest.apply(movie_id_vectors)).reset_index()
ab

,userId,0
0,4.0,"[4.0, 4.0, 0, 2.0, 4.5, 0, 4.0, 3.5, 0, 0, 4.0..."
1,10.0,"[5.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,14.0,"[4.5, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,15.0,"[4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
4,22.0,"[4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...
95,399.0,"[4.0, 0, 3.0, 0, 4.0, 4.0, 4.0, 5.0, 0, 0, 0, ..."
96,400.0,"[3.0, 0, 0, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 4.0, ..."
97,402.0,"[4.5, 0, 0, 0, 0, 3.5, 3.0, 0, 0, 0, 0, 0, 0, ..."
98,408.0,"[3.0, 0, 0, 0, 0, 0, 0, 0, 0, 4.0, 3.0, 3.0, 0..."


In [29]:
ab.to_csv('ab.csv')

In [30]:
import numpy as np
from numpy.linalg import norm

In [31]:
mat = np.empty((len(ab),len(ab)))
mat

array([[9.92449857e-312, 9.92448254e-312, 1.06099790e-312, ...,
        6.79038653e-313, 9.33678148e-313, 1.01855798e-312],
       [6.79038653e-313, 9.33678148e-313, 1.01855798e-312, ...,
        9.33678148e-313, 1.01855798e-312, 6.79038653e-313],
       [9.33678148e-313, 1.01855798e-312, 6.79038653e-313, ...,
        9.33678148e-313, 1.01855798e-312, 6.79038653e-313],
       ...,
       [6.31860554e-320, 6.32058181e-320, 6.32206400e-320, ...,
        6.63530162e-320, 6.63727789e-320, 6.64172448e-320],
       [6.64271261e-320, 6.64370074e-320, 6.64617107e-320, ...,
        7.00881525e-320, 7.01128558e-320, 7.01227371e-320],
       [7.01770843e-320, 7.01919063e-320, 7.02462535e-320, ...,
        7.37985855e-320, 7.38183481e-320, 7.38331701e-320]])

In [32]:
def normmmm(vec):
        B = np.array(list(vec))
        return norm(B)

normb = ab[0].apply(normmmm)
normb

0     85.599357
1     39.849718
2     39.281039
3     43.657760
4     13.379088
        ...    
95    45.847574
96    13.304135
97    65.124880
98    52.287666
99    10.594810
Name: 0, Length: 100, dtype: float64

In [33]:
for i in range(len(ab)):
    A = np.array(list(ab.iloc[i][0]))
    norma = normb[i]
    for j in range(i,len(ab)):
        B = np.array(list(ab.iloc[j][0]))
        mat[i][j] = np.dot(A,B)/(norma*normb[j])
        mat[j][i] = mat[i][j]
mat

array([[1.        , 0.13280134, 0.19219721, ..., 0.45258464, 0.19382051,
        0.12680435],
       [0.13280134, 1.        , 0.07698016, ..., 0.16068076, 0.32635059,
        0.12316432],
       [0.19219721, 0.07698016, 1.        , ..., 0.20698366, 0.09226288,
        0.1808133 ],
       ...,
       [0.45258464, 0.16068076, 0.20698366, ..., 1.        , 0.13714207,
        0.16087289],
       [0.19382051, 0.32635059, 0.09226288, ..., 0.13714207, 1.        ,
        0.09386656],
       [0.12680435, 0.12316432, 0.1808133 , ..., 0.16087289, 0.09386656,
        1.        ]])

In [34]:
mat.shape

(100, 100)

In [35]:
filename = 'array.csv'

# Save the array to a CSV file
np.savetxt(filename, mat, delimiter=',')

In [36]:
# Create mapping of genres
genre_map = dict(zip(sorted(new_new_df['genres'].unique()), range(len(new_new_df['genres'].unique()))))

In [52]:
genre_map

{'Action|Adventure': 0,
 'Action|Adventure|Comedy': 1,
 'Action|Adventure|Comedy|Crime': 2,
 'Action|Adventure|Comedy|Fantasy': 3,
 'Action|Adventure|Comedy|Romance|Thriller': 4,
 'Action|Adventure|Comedy|Sci-Fi': 5,
 'Action|Adventure|Drama': 6,
 'Action|Adventure|Drama|Fantasy': 7,
 'Action|Adventure|Drama|War': 8,
 'Action|Adventure|Fantasy': 9,
 'Action|Adventure|Sci-Fi': 10,
 'Action|Adventure|Sci-Fi|IMAX': 11,
 'Action|Adventure|Sci-Fi|Thriller': 12,
 'Action|Adventure|Sci-Fi|Thriller|IMAX': 13,
 'Action|Adventure|Thriller': 14,
 'Action|Comedy': 15,
 'Action|Comedy|Crime|Drama': 16,
 'Action|Comedy|Crime|Thriller': 17,
 'Action|Comedy|Sci-Fi': 18,
 'Action|Crime': 19,
 'Action|Crime|Drama': 20,
 'Action|Crime|Drama|Thriller': 21,
 'Action|Crime|Thriller': 22,
 'Action|Drama': 23,
 'Action|Drama|Romance': 24,
 'Action|Drama|Sci-Fi|Thriller': 25,
 'Action|Drama|Thriller': 26,
 'Action|Drama|War': 27,
 'Action|Horror|Sci-Fi|Thriller': 28,
 'Action|Romance|Thriller': 29,
 'Action|Sc

In [37]:
#function to create SVNS
def SVNS(df):
    temp = [ [0,0,0] for i in range(new_new_df['genres'].nunique())]
    df = df.reset_index()
    for g in df['genres'].unique():
        #print(g)
        rat = df[df['genres'] == g]
        gg = len(rat[rat["rating"]>=4])
        n = len(rat[rat["rating"]>2])-gg
        b = len(rat[rat["rating"]<=2])

        temp[genre_map[g]]=[gg,n,b]
    return temp        

In [38]:
u_g_sv = pd.DataFrame(gbtest.apply(SVNS)).reset_index()
u_g_sv

,userId,0
0,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [..."
1,10.0,"[[1, 0, 0], [0, 0, 1], [0, 0, 0], [0, 0, 0], [..."
2,14.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 1, 0], [..."
3,15.0,"[[0, 1, 0], [2, 0, 0], [0, 0, 0], [0, 0, 0], [..."
4,22.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."
...,...,...
95,399.0,"[[2, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."
96,400.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."
97,402.0,"[[2, 1, 0], [2, 1, 0], [0, 0, 1], [1, 0, 0], [..."
98,408.0,"[[0, 2, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."


In [39]:
test_crossjoin = u_g_sv
test_crossjoin['key'] = 1
result = pd.merge(test_crossjoin, test_crossjoin, on ='key').drop(["key"],axis=1)
result

,userId_x,0_x,userId_y,0_y
0,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [..."
1,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",10.0,"[[1, 0, 0], [0, 0, 1], [0, 0, 0], [0, 0, 0], [..."
2,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",14.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 1, 0], [..."
3,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",15.0,"[[0, 1, 0], [2, 0, 0], [0, 0, 0], [0, 0, 0], [..."
4,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",22.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."
...,...,...,...,...
9995,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",399.0,"[[2, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."
9996,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",400.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."
9997,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",402.0,"[[2, 1, 0], [2, 1, 0], [0, 0, 1], [1, 0, 0], [..."
9998,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",408.0,"[[0, 2, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."


In [40]:
mat

array([[1.        , 0.13280134, 0.19219721, ..., 0.45258464, 0.19382051,
        0.12680435],
       [0.13280134, 1.        , 0.07698016, ..., 0.16068076, 0.32635059,
        0.12316432],
       [0.19219721, 0.07698016, 1.        , ..., 0.20698366, 0.09226288,
        0.1808133 ],
       ...,
       [0.45258464, 0.16068076, 0.20698366, ..., 1.        , 0.13714207,
        0.16087289],
       [0.19382051, 0.32635059, 0.09226288, ..., 0.13714207, 1.        ,
        0.09386656],
       [0.12680435, 0.12316432, 0.1808133 , ..., 0.16087289, 0.09386656,
        1.        ]])

In [41]:
def mapping_userid_xandy(row):
    return mat[user_id_map[row['userId_x']],user_id_map[row['userId_y']]]

In [42]:
result["cosin"] = result.apply(mapping_userid_xandy,axis = 1)
result

,userId_x,0_x,userId_y,0_y,cosin
0,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",1.000000
1,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",10.0,"[[1, 0, 0], [0, 0, 1], [0, 0, 0], [0, 0, 0], [...",0.132801
2,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",14.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 1, 0], [...",0.192197
3,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",15.0,"[[0, 1, 0], [2, 0, 0], [0, 0, 0], [0, 0, 0], [...",0.246851
4,4.0,"[[4, 1, 1], [1, 2, 1], [0, 0, 1], [2, 0, 2], [...",22.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",0.072037
...,...,...,...,...,...
9995,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",399.0,"[[2, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",0.178076
9996,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",400.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",0.063850
9997,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",402.0,"[[2, 1, 0], [2, 1, 0], [0, 0, 1], [1, 0, 0], [...",0.160873
9998,412.0,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",408.0,"[[0, 2, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",0.093867


In [43]:
import scipy

In [44]:
len(result['0_x'][0])

100

In [45]:
def genre_wise_cosinesimilarity(row):
    temp = [ 0 for i in range(len(result['0_x'][0]))]
    for j in range(0,len(row[1])):
        temp[j] = (scipy.spatial.distance.cosine(row[1][j],row[3][j]))
    return temp

In [46]:
result1 = pd.DataFrame()
result1['User1'] = result['userId_x']
result1['User2'] = result['userId_y']
result1['genre_based_cosinesimilarity'] = result.apply(genre_wise_cosinesimilarity,axis=1)
result1["Cosine_between_user"] = result['cosin']
result1

d:\Python\lib\site-packages\scipy\spatial\distance.py:622: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


,User1,User2,genre_based_cosinesimilarity,Cosine_between_user
0,4.0,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.000000
1,4.0,10.0,"[0.057190958417936755, 0.591751709536137, 0, 0...",0.132801
2,4.0,14.0,"[0, 0, 0, 0.5, 0, 0.29289321881345254, 0, 0, 0...",0.192197
3,4.0,15.0,"[0.7642977396044842, 0.591751709536137, 0, 0, ...",0.246851
4,4.0,22.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.072037
...,...,...,...,...
9995,412.0,399.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5527864...",0.178076
9996,412.0,400.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.063850
9997,412.0,402.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2155354594472...",0.160873
9998,412.0,408.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0...",0.093867


In [47]:
r_test = pd.DataFrame(result1['genre_based_cosinesimilarity'])
r_test = pd.DataFrame(r_test.genre_based_cosinesimilarity.tolist(), index= r_test.index)
r_test["Cosine_between_user"] = result['cosin']
r_test

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Cosine_between_user
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000
1,0.057191,0.591752,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.013606,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.552786,0.132801
2,0.000000,0.000000,0.0,0.5,0.0,0.292893,0.000000,0.0,0.0,0.0,...,0.012237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105573,0.192197
3,0.764298,0.591752,0.0,0.0,0.0,0.000000,0.105573,0.0,1.0,0.0,...,0.044221,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.246851
4,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.072037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.178076
9996,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.063850
9997,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.160873
9998,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.093867


In [48]:
r_test.to_csv("training_data_x.csv")

In [ ]:
import pandas as pd
train_new = pd.read_csv("/content/gdrive/My Drive/training_data_x.csv")

In [ ]:
#comparing all the machine learning models using pycaret
from pycaret.regression import *
exp_reg101 = setup(data = train_new, target = 'Cosine_between_user',session_id = 123)
best = compare_models()

In [ ]:
#training catboost model since pycaret doesnt test it
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import time

# Assign X and y from the dataset
X = train_new.drop(columns=['Cosine_between_user'])
y = train_new['Cosine_between_user']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the CatBoostRegressor model
start_time = time.time()
model = CatBoostRegressor()
model.fit(X_train, y_train)
end_time = time.time()

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
time_taken = end_time - start_time

# Print the evaluation metrics
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)
print("RMSLE:", rmsle)
print("MAPE:", mape)
print("Time taken:", time_taken)

In [ ]:
#START OF BASELINE MODEL

In [55]:
#function to find average rating
def avg_rating(df):
    temp = [ 0 for i in range(new_new_df['genres'].nunique())]
    df = df.reset_index()
    sum = 0
    for g in df['genres'].unique():
        #print(g)
        rat = df[df['genres'] == g]
        avg = rat["rating"].mean()     
        temp[genre_map[g]]= avg
    return temp     

In [56]:
u_g_avg = pd.DataFrame(gbtest.apply(avg_rating)).reset_index()
u_g_avg

,userId,0
0,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5..."
1,10.0,"[5.0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 0, ..."
2,14.0,"[0, 0, 0, 3.75, 0, 4.0, 4.0, 0, 4.0, 0, 3.625,..."
3,15.0,"[3.0, 5.0, 0, 0, 0, 0, 4.0, 5.0, 2.5, 0, 2.5, ..."
4,22.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
95,399.0,"[4.0, 0, 0, 0, 0, 4.0, 4.333333333333333, 0, 4..."
96,400.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0..."
97,402.0,"[3.6666666666666665, 3.8333333333333335, 2.0, ..."
98,408.0,"[3.0, 0, 0, 0, 0, 0, 3.0, 0, 0, 2.0, 2.4, 0, 3..."


In [58]:
test_crossjoin1 = u_g_avg
test_crossjoin1['key'] = 1
resultx = pd.merge(test_crossjoin1, test_crossjoin1, on ='key').drop(["key"],axis=1)
resultx

,userId_x,0_x,userId_y,0_y
0,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5..."
1,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",10.0,"[5.0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 0, ..."
2,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",14.0,"[0, 0, 0, 3.75, 0, 4.0, 4.0, 0, 4.0, 0, 3.625,..."
3,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",15.0,"[3.0, 5.0, 0, 0, 0, 0, 4.0, 5.0, 2.5, 0, 2.5, ..."
4,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",22.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
9995,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",399.0,"[4.0, 0, 0, 0, 0, 4.0, 4.333333333333333, 0, 4..."
9996,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",400.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0..."
9997,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",402.0,"[3.6666666666666665, 3.8333333333333335, 2.0, ..."
9998,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",408.0,"[3.0, 0, 0, 0, 0, 0, 3.0, 0, 0, 2.0, 2.4, 0, 3..."


In [59]:
resultx["cosin"] = result.apply(mapping_userid_xandy,axis = 1)
resultx

,userId_x,0_x,userId_y,0_y,cosin
0,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",1.000000
1,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",10.0,"[5.0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 0, ...",0.132801
2,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",14.0,"[0, 0, 0, 3.75, 0, 4.0, 4.0, 0, 4.0, 0, 3.625,...",0.192197
3,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",15.0,"[3.0, 5.0, 0, 0, 0, 0, 4.0, 5.0, 2.5, 0, 2.5, ...",0.246851
4,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 3.5...",22.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.072037
...,...,...,...,...,...
9995,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",399.0,"[4.0, 0, 0, 0, 0, 4.0, 4.333333333333333, 0, 4...",0.178076
9996,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",400.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0...",0.063850
9997,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",402.0,"[3.6666666666666665, 3.8333333333333335, 2.0, ...",0.160873
9998,412.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5.0, 0, 3.5, 0,...",408.0,"[3.0, 0, 0, 0, 0, 0, 3.0, 0, 0, 2.0, 2.4, 0, 3...",0.093867


In [60]:
resultx['0_x'][0]

[3.6666666666666665,
 3.25,
 2.0,
 3.0,
 3.75,
 3.5,
 5.0,
 4.0,
 4.75,
 3.642857142857143,
 3.6944444444444446,
 4.0,
 3.9,
 2.5,
 3.88,
 2.9,
 4.1,
 2.8333333333333335,
 2.9,
 3.5,
 4.375,
 4.4,
 4.0588235294117645,
 4.1,
 3.75,
 3.3333333333333335,
 3.4444444444444446,
 4.4375,
 3.4166666666666665,
 3.5,
 3.1875,
 3.40625,
 3.7083333333333335,
 4.0,
 3.75,
 2.5,
 3.5,
 2.7142857142857144,
 4.5,
 2.8333333333333335,
 3.75,
 4.0,
 3.6666666666666665,
 4.0,
 3.875,
 2.25,
 0,
 1.5,
 1.0,
 3.5,
 2.901639344262295,
 2.6,
 3.8333333333333335,
 4.166666666666667,
 4.5,
 2.5833333333333335,
 3.0,
 3.0,
 2.45,
 3.5,
 3.25,
 2.7,
 3.3333333333333335,
 0,
 2.911764705882353,
 2.625,
 1.5,
 3.9583333333333335,
 3.0,
 4.25,
 5.0,
 4.5,
 4.333333333333333,
 4.2,
 4.125,
 3.2857142857142856,
 4.25,
 5.0,
 0.5,
 0,
 1.5,
 0,
 4.5,
 3.2,
 3.388888888888889,
 4.0,
 0,
 4.3,
 3.6,
 3.566666666666667,
 3.0,
 4.357142857142857,
 0,
 2.25,
 2.0,
 3.6,
 3.0,
 4.5,
 5.0,
 3.9166666666666665]

In [65]:
import math

In [73]:
def genre_wise_pearsonsimilarity(row):
    temp = [ 0 for i in range(len(resultx['0_x'][0]))]
    for j in range(0,len(row)):
        #temp[j] = (np.corrcoef(row['0_x'][j], row['0_y'][j])[0, 1])
        temp[j] = (row['0_x'][j] + row['0_y'][j])/2
    return temp

In [74]:
result_avg = pd.DataFrame()
result_avg['User1'] = resultx['userId_x']
result_avg['User2'] = resultx['userId_y']


In [75]:
result_avg['genre_based_cosinesimilarity'] = resultx.apply(genre_wise_pearsonsimilarity,axis=1)

In [76]:
result_avg["Cosine_between_user"] = resultx['cosin']
result_avg

,User1,User2,genre_based_cosinesimilarity,Cosine_between_user
0,4.0,4.0,"[3.6666666666666665, 3.25, 2.0, 3.0, 3.75, 0, ...",1.000000
1,4.0,10.0,"[4.333333333333333, 2.625, 1.0, 1.5, 1.875, 0,...",0.132801
2,4.0,14.0,"[1.8333333333333333, 1.625, 1.0, 3.375, 1.875,...",0.192197
3,4.0,15.0,"[3.333333333333333, 4.125, 1.0, 1.5, 1.875, 0,...",0.246851
4,4.0,22.0,"[1.8333333333333333, 1.625, 1.0, 1.5, 1.875, 0...",0.072037
...,...,...,...,...
9995,412.0,399.0,"[2.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",0.178076
9996,412.0,400.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",0.063850
9997,412.0,402.0,"[1.8333333333333333, 1.9166666666666667, 1.0, ...",0.160873
9998,412.0,408.0,"[1.5, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0,...",0.093867


In [78]:
r_test = pd.DataFrame(result_avg['genre_based_cosinesimilarity'])
r_test = pd.DataFrame(r_test.genre_based_cosinesimilarity.tolist(), index= r_test.index)
r_test["Cosine_between_user"] = result['cosin']
r_test

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Cosine_between_user
0,3.666667,3.250000,2.0,3.000,3.750,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000000
1,4.333333,2.625000,1.0,1.500,1.875,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.132801
2,1.833333,1.625000,1.0,3.375,1.875,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.192197
3,3.333333,4.125000,1.0,1.500,1.875,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.246851
4,1.833333,1.625000,1.0,1.500,1.875,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.072037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.000000,0.000000,0.0,0.000,0.000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.178076
9996,0.000000,0.000000,0.0,0.000,0.000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.063850
9997,1.833333,1.916667,1.0,2.000,0.000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.160873
9998,1.500000,0.000000,0.0,0.000,0.000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.093867


In [79]:
r_test.to_csv("training_data_baseline.csv")

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data = train_new, target = 'Cosine_between_user',session_id = 123)
best_models = compare_models(n_select=21)

In [ ]:
#training catboost regressor since pycaret doesnt use it
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import time

# Assign X and y from the dataset
X = train_new.drop(columns=['Cosine_between_user'])
y = train_new['Cosine_between_user']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the CatBoostRegressor model
start_time = time.time()
model = CatBoostRegressor()
model.fit(X_train, y_train)
end_time = time.time()

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
time_taken = end_time - start_time

# Print the evaluation metrics
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-squared:", r2)
print("RMSLE:", rmsle)
print("MAPE:", mape)
print("Time taken:", time_taken)
